In [1]:
import argparse
import os
import torch
from utils.utils import read_yaml
from utils.dataloader_factory import create_dataloader
import warnings
from eval_utils import evaluation
from model import CHIEF_survival
warnings.filterwarnings("ignore")


def load_model(cfg):
    model = CHIEF_survival(n_classes=cfg.Data.n_classes, **cfg.Model)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.train()
    return model

def eval_single(i, model, dataloader, result_dir, cfg, study):
    res_df,cindex = evaluation(i, model, dataloader, result_dir, cfg)
    print('cindex:', cindex)
    median = res_df['risk'].median()
    res_df.loc[res_df['risk']<=median, 'pred'] = 0
    res_df.loc[res_df['risk']>median, 'pred'] = 1
    return res_df

def eval_pipeline(dataset_name, cfg, result_dir):
    os.makedirs(result_dir, exist_ok=True)
    res_df_total = pd.DataFrame()
    dataloader = create_dataloader(0, dataset_name, cfg, result_dir)
    model.load_state_dict(
        torch.load(ckpt_dir))
    res_df = eval_single(0, model, dataloader, result_dir, cfg, dataset_name)
    res_df.to_csv('./results/evaluation/tcga_rcc/prediction.csv')

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cfg = read_yaml('./cfg_rcc_cox_dss.yaml')
dataset_name = 'val_set'
model = load_model(cfg)
result_dir1 = os.path.join(cfg.General.result_dir, 
                            'evaluation', cfg.Data.project)
ckpt_dir = os.path.join('./weights/best_checkpoints/s_0_checkpoint.pt')
eval_pipeline(dataset_name, cfg, result_dir1)

cindex: 0.9


In [3]:
import pandas as pd
df = pd.read_csv('./results/evaluation/tcga_rcc/prediction.csv')
print(df[['patient_id', 'status', 'risk', 'pred']])

     patient_id  status      risk  pred
0  TCGA-B0-5691     0.0 -1.265317   0.0
1  TCGA-B0-5713     0.0 -0.398486   0.0
2  TCGA-BP-4167     0.0 -0.328713   0.0
3  TCGA-BP-4329     1.0  0.913136   1.0
4  TCGA-BP-4335     1.0  0.405751   1.0
5  TCGA-CJ-4885     0.0  0.262248   1.0
6  TCGA-KL-8339     1.0 -0.144757   1.0
7  TCGA-KN-8429     0.0 -1.714606   0.0
8  TCGA-P4-A5E6     0.0 -0.862301   0.0
9  TCGA-P4-AAVL     1.0  2.203031   1.0
